Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (52909, 28, 28) (52909,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (52909, 28, 28, 1) (52909, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.799332
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.335764
Minibatch accuracy: 43.8%
Validation accuracy: 58.9%
Minibatch loss at step 100: 1.706716
Minibatch accuracy: 50.0%
Validation accuracy: 71.2%
Minibatch loss at step 150: 0.697959
Minibatch accuracy: 81.2%
Validation accuracy: 62.1%
Minibatch loss at step 200: 1.148394
Minibatch accuracy: 62.5%
Validation accuracy: 75.5%
Minibatch loss at step 250: 0.900576
Minibatch accuracy: 68.8%
Validation accuracy: 69.1%
Minibatch loss at step 300: 0.415743
Minibatch accuracy: 87.5%
Validation accuracy: 77.5%
Minibatch loss at step 350: 0.346722
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 400: 0.356201
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.402863
Minibatch accuracy: 87.5%
Validation accuracy: 79.9%
Minibatch loss at step 500: 0.889582
Minibatch accuracy: 68.8%
Validation accuracy: 80.3%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [8]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size//4  * image_size//4  * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

  # Model.
  def model(set):
      pool   = tf.nn.max_pool(set, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
      hidden = tf.nn.relu(pool + layer1_biases)
      pool   = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
      hidden = tf.nn.relu(pool + layer2_biases)
      shape  = hidden.get_shape().as_list()
      reshape= tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
      hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
      return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.819203
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.294145
Minibatch accuracy: 12.5%
Validation accuracy: 9.0%
Minibatch loss at step 100: 2.216978
Minibatch accuracy: 31.2%
Validation accuracy: 10.0%
Minibatch loss at step 150: 2.287304
Minibatch accuracy: 12.5%
Validation accuracy: 13.9%
Minibatch loss at step 200: 2.319241
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 250: 2.308621
Minibatch accuracy: 0.0%
Validation accuracy: 16.9%
Minibatch loss at step 300: 2.305853
Minibatch accuracy: 0.0%
Validation accuracy: 14.1%
Minibatch loss at step 350: 2.145487
Minibatch accuracy: 31.2%
Validation accuracy: 11.3%
Minibatch loss at step 400: 2.124410
Minibatch accuracy: 31.2%
Validation accuracy: 11.3%
Minibatch loss at step 450: 2.082942
Minibatch accuracy: 37.5%
Validation accuracy: 23.6%
Minibatch loss at step 500: 2.178521
Minibatch accuracy: 12.5%
Validation accuracy: 16.6%
Minib

Eek, terrible accuracy due to the instructions deliberately specifying replacement of conv functions with pool instead of adding them.

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [12]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Or death by Matrix manipulation.
 
 #Conv, hidden, pool
  layer1_weights = tf.Variable(tf.truncated_normal(
      [5, 5, 1, 32], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([32]))
  
 #Conv, hidden, pool
  layer2_weights = tf.Variable(tf.truncated_normal(
      [5, 5, 32, 64], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[64]))
 
 #Hidden, drop
  layer3_weights = tf.Variable(tf.truncated_normal(
      [7*7*64, 1024], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[1024]))
 
 #Weights*x + bias (aka logreg)
  layer4_weights = tf.Variable(tf.truncated_normal(
      [1024, 10], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[10]))
 
  # Model.
  def model(data, train):
    #Layer1
      conv   = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
      hidden = tf.nn.relu(conv + layer1_biases)
      pool   = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID') 
    #Layer2
      conv   = tf.nn.conv2d(pool, layer2_weights, [1, 1, 1, 1], padding='SAME') 
      hidden = tf.nn.relu(conv + layer2_biases)
      pool   = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='VALID')
    #Reshaping to proper depth, flattened
      reshape= tf.reshape(pool, [-1, 7*7*64])
    #Layer3
      hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
      if (train):
        hidden = tf.nn.dropout(hidden, 0.5)
    #Layer4
      return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset, True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = 0.0005
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, False))

In [11]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2000 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 36.940403
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 2000: 0.378186
Minibatch accuracy: 87.5%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 0.704803
Minibatch accuracy: 81.2%
Validation accuracy: 86.8%
Minibatch loss at step 6000: 0.361458
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 8000: 0.231032
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 10000: 0.547983
Minibatch accuracy: 81.2%
Validation accuracy: 89.1%
Minibatch loss at step 12000: 0.118086
Minibatch accuracy: 100.0%
Validation accuracy: 89.4%
Minibatch loss at step 14000: 0.507204
Minibatch accuracy: 87.5%
Validation accuracy: 90.1%
Minibatch loss at step 16000: 0.141231
Minibatch accuracy: 93.8%
Validation accuracy: 90.3%
Minibatch loss at step 18000: 0.204157
Minibatch accuracy: 87.5%
Validation accuracy: 90.4%
Minibatch loss at step 20000: 0.220880
Minibatch accuracy: 93.8%
Validatio

 95.6%. Given more steps (I expect a further 10,000 would do the trick given the low learning rate), we could very quickly approach and match the existing best performance (99.7%). Either way, by adding extra layers, dropouts, and pooling we were able to improve accuracy by 6%.